## GPT+RAG

In [1]:
import pickle
import numpy as np
from datetime import datetime
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from sentence_transformers.similarity_functions import cos_sim
import os

c:\Users\adsky\OneDrive\바탕 화면\DATA_CAMP\Develops\24s_nlfp_project\nlfp_project\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
class EnhancedChatbot:
    def __init__(self, api_key, model_name, db_path):
        self.client = OpenAI(api_key=api_key)
        self.embedding_model = SentenceTransformer(model_name)
        self.db_path = db_path
        self.load_database()
        self.current_conversation = []

    def load_database(self):
        with open(self.db_path, 'rb') as f:
            data = pickle.load(f)
        self.dataset = data['dataset']
        self.embeddings = data['embeddings']


        #     if not isinstance(self.embeddings, torch.Tensor):
        #         self.embeddings = torch.tensor(self.embeddings)
        #     print(f"Database loaded successfully from {self.db_path}")
        # except (FileNotFoundError, EOFError, KeyError) as e:
        #     print(f"Error loading database from {self.db_path}: {e}")
        #     print("Initializing empty database.")
        #     self.dataset = []
        #     self.embeddings = torch.tensor([])

    def save_database(self):
        # 데이터베이스 저장
        data_to_save = {
            'dataset': self.dataset,
            'embeddings': self.embeddings
        }
        with open(self.db_path, 'wb') as f:
            pickle.dump(data_to_save, f)
        print(f"Database saved successfully to {self.db_path}")

    def get_relevant_documents(self, query, k=10):
        # 쿼리와 가장 관련성 높은 상위 k개의 문서 검색
        query_embedding = self.embedding_model.encode(query)
        similarities = cos_sim(self.embeddings, query_embedding)
        top_k_indices = similarities.squeeze().argsort(descending=True)[:k]
        return [self.dataset[i] for i in top_k_indices]

    def get_response(self, query):
        # GPT API를 사용하여 응답 생성
        relevant_docs = self.get_relevant_documents(query)
        context = "\n\n".join([f"{doc['title']}\n{doc['text']}" if isinstance(doc, dict) else str(doc) for doc in relevant_docs])
        # context = "\n\n".join([f"{doc['title']}\n{doc['text']}" for doc in relevant_docs])

        messages = [
            {"role": "system", "content": "자연스럽게 말하기"},
            {"role": "user", "content": f"Context: {context}\n\nQuery: {query}"}
        ]

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )

        response_content = response.choices[0].message.content

        # 대화 내용 저장
        self.current_conversation.append((query, response_content))

        return response_content

    def save_conversation(self, query, response):
        # 대화 내용을 시각과 함께 데이터베이스에 저장
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        conversation_text = "\n".join([f"User: {query}\nAssistant: {response}" for query, response in self.current_conversation])
        conversation = {
            'title': f"Conversation at {timestamp}",
            'text': conversation_text
        }

        self.dataset.append(conversation)
        embedding = self.embedding_model.encode(f"{conversation['title']}\n{conversation['text']}")
        # self.embeddings.append(embedding)
        if self.embeddings.size == 0:
          self.embeddings = np.array([embedding])
        else:
          self.embeddings = np.vstack((self.embeddings, embedding))


        self.save_database()
        # self.current_conversation=[]


In [7]:
# 챗봇 사용 예시
if __name__ == "__main__":
    api_key = os.getenv('api_key')
    model_name = "jhgan/ko-sbert-multitask"
    db_path = "./data/dataset_and_embeddings_text_test.pkl"
    today_conversation =[]

    # 데이터베이스 파일의 존재 여부 확인
    if not os.path.exists(db_path):
        print(f"Warning: Database file not found at {db_path}")
        print("Please make sure the file exists in your Google Drive.")
        # 여기서 초기 데이터베이스 생성 로직을 추가할 수 있습니다.
    else:
        print(f"Database file found at {db_path}")

    chatbot = EnhancedChatbot(api_key, model_name, db_path) # class 가지고 옴

    while True:
        user_input = input("User: ")
        if user_input.lower() == 'exit':
            print("Chatbot: 대화를 종료합니다.")
            #요악하는 거 넣기
            # Accumulate conversation in today_conversation

            break
        response = chatbot.get_response(user_input)
        today_conversation.append((user_input, response))
        print("User:", user_input)
        print("Chatbot:", response)



    # # 대화 저장
    # chatbot.save_conversation(user_input, response)

Database file found at ./data/dataset_and_embeddings_text_test.pkl
User: 안녕
Chatbot: 안녕하세요! 오늘 기분은 어떠세요? 운동을 시작할 준비가 되셨나요?
User: 
Chatbot: 어떤 도움이 필요하신가요? 질문이나 궁금한 점이 있으시면 말씀해 주세요.
Chatbot: 대화를 종료합니다.


In [41]:
today_conversation

[('농구할래',
  '내가 왜 농구에 그렇게 빠지게 되었는지 알겠어. 농구를 정말 좋아하는 게 느껴진다! 그러면 오늘은 그냥 가볍게 슛 연습이나 드리블 같은 것만 해보는 건 어때? 꼭 강도 높은 운동이 아니어도 좋으니까, 즐기면서 꾸준히 하는 게 중요해.\n\n그리고 또 보람찬 시간을 보낼 수 있을 거야. 힘들 땐 쉬어도 괜찮으니까, 너무 무리하지 말고 네 몸 상태에 맞춰서 해보자! 😊🏀✨'),
 ('음 피자 먹을래',
  '농구하고 싶다니 좋은 생각이야! 어제도 농구 이야기 나왔었잖아. 지금 바로 나가서 운동하면 기분도 상쾌하고, 체력도 더 좋아질 거야. 벌써 클럽도 있고, 자주 농구하는 것 같아서 참 보기 좋아. \n\n오늘 농구할 때 어떤 스킬을 연습할 계획이야? 드리블이나 슛, 아니면 패스 같은 부분을 더 집중적으로 연습해보는 건 어때? 클럽 친구들과 같이 연습하면 더 재미있고 효과적이니까 아주 좋은 시간이 될 거야. 다치지 않도록 조심하고, 파이팅! 🏀😊\n\n농구 끝나고 어떤 메뉴로 먹을 계획이야? 농구 후에는 좋은 영양 섭취가 중요하니까 꼭 챙겨 먹고!')]

In [28]:
messages = [
{"role": "system", "content": "사용자의 말을 간단하게 요약해줘. 격려해주는 뉘앙스로 대답해줘. 대답 후에는 비슷한 소재로 대화를 이어나가줘. 어느정도 대화가 진행되면 마무리해줘. 유저 기본 정보 (내가 어떤 사람인지에 대한 기본 정보 제공) 안녕 나는 24살 남자고, 키 180cm에 몸무게는 75kg야 목표 운동 횟수는 주 4회이고, 운동의 목적은 건강을 유지하기 위해서야 운동을 시작한지는 3개월 되었고, 함께 운동하는 친구나 가족은 없고 혼자서 운동하고 있어. 운동을 지속적으로 하고 싶은데, 혼자서 하다보니까 잘 안돼서 걱정이야.\n원하는 답변 방식\n너는 운동을 20년 동안 꾸준히 하였고, 10년 이상의 경력을 가지고 있는 헬스 트레이너야. 영양학 및 해부학에 대해서 박사 수준의 지식을 가지고 있어. 대답을 할 때 상대방이 운동을 지속적으로 할 수 있게 운동에 대한 기본적인 정보를 알려주는 것도 중요한데, 초보자들을 위한 심리적인 지지와 동기부여가 되도록 공감하면서 말을 해줬으면 좋겠어.\n아래가 옳은 예시야 안녕 나 운동 방금 끝냈어 고생했네 오늘은 어떤 운동했어? 달리기 30분하고 턱걸이랑 푸쉬업 연습 각각 30분했어 운동 많이했네! 고생 많았어. 이번 주 운동 몇 번 갔지? 이번 주 오늘 포함해서 3번 갔어 오 이번 주는 목표 횟수 달성했네 고생 많았어! 운동할 때 뭐가 가장 힘들었어? 오늘 턱걸이 3개 하는게 목표였는데, 2개밖에 못해서 속상했어 괜찮아 다음에 하면 되지 그럼 뭐 할때 가장 기분이 좋았어? 푸쉬업 10개가 목표였는데, 9개 할 때 힘들어서 포기할까 했는데 꾹 참고 했어 들었을텐데 했구나 고생 많았어! 이제 뭐할거야? 이제 집가서 밥이랑 닭가슴살 먹으려구 그래 오늘도 고생 많았어! 다음 운동할 때 또 얘기하자\n이 아래는 이렇게 대답하지 않았으면 하는 잘못된 예시야 안녕하세요! 운동을 이제 시작하신 걸 환영합니다. 저는 20년간 운동을 꾸준히 해온 헬스 트레이너로, 영양학과 해부학에 대해 깊이 있는 지식을 가지고 있습니다. 운동에 대한 기본적인 정보부터 하나씩 설명해 드리겠습니다. 시작해볼까요? 운동의 기본 원리\n운동의 종류: • 유산소 운동 (Cardio): 걷기, 달리기, 자전거 타기 등 심박수를 올려서 심폐 기능을 강화하는 운동입니다. • 무산소 운동 (Strength Training): 웨이트 리프팅, 스쿼트, 푸쉬업 등 근육을 강화하고 근력을 키우는 운동입니다. • 스트레칭 (Stretching): 유연성을 높이고 부상을 예방하는 데 도움이 됩니다.\n운동의 기본 구성: • 준비 운동 (Warm-up): 본 운동 전에 체온을 올리고 근육을 풀어주는 단계입니다. 5-10분 정도 가벼운 유산소 운동이나 스트레칭을 추천합니다. • 본 운동 (Main Workout): 유산소 운동과 무산소 운동을 포함한 주된 운동 시간입니다. 초보자의 경우 각 운동을 20-30분씩, 주 3-4회 정도가 적당합니다. • 정리 운동 (Cool-down): 본 운동 후에 심박수를 서서히 낮추고 근육을 이완시키는 단계입니다. 5-10분 정도의 가벼운 스트레칭이 좋습니다.\n단문장으로 말하기\n목적: 복잡한 정보를 명확하게 전달하기 위해 문장을 간결하게 만드는 것.\n잘못된 예시:\n민호랑 경희는 커플이고 민호에겐 동수라는 친구가 있었는데 동수가 경희한테 고백했어.\n옳은 예시:\n민호랑 경희는 커플이야.\n민호에겐 동수라는 친구가 있었어.\n근데 동수가 경희한테 고백했어.\n경청하기\n목적: 상대방의 말을 잘 듣고 반응하여 대화의 흐름을 원활하게 만드는 것.\n대체로 사람은 듣는 것보다 말하기를 좋아한다. 그러니 말하고 싶은 욕구를 조금만 참고 상대방 말에 귀를 기울여 보자. 경청하면 대화가 훨씬 수월해진다. 당신이 노력하지 않아도 상대가 알아서 대화를 이어가기 때문이다. 그런데 재밌는 사실은 계속 떠든 건 상대방이지만 정작 그들은 당신이 말을 잘한다고 생각한다는 것이다. 상대가 자신의 관심사에 관해서 이야기하고 있으면 일단 경청하자.\n잘못된 예시:\n“잠깐만, 그 얘기 나중에 하고, 이거 먼저 말해줄게.”\n“그거 별로 어렵지 않잖아요.”\n“아, 그래요. 근데 저도 할 일이 너무 많아요.”\n옳은 예시:\n“와 정말요?”\n“전혀 생각 못 했어요.”\n“어떻게 이런 것까지 알고 계세요?”\n“정말 대단해요.”\n사소한 의견에 동의하기\n목적: 상대방과의 공감을 형성하고 대화의 흐름을 부드럽게 만들기 위해.\n잘못된 예시: 간이 조금 센거 같지 않아? 아닌데? 맛집이구만\n옳은 예시:\n간이 조금 센 거 같지 않아?\n그러게.\n간이 좀 세네.\n그래도 맛은 있다.\n질문에 답하고 한번 더 물어보기\n목적: 형식적인 대답을 피하고, 대화를 더 풍부하게 만들어 상호작용을 활성화시키기 위해.\n잘못된 예시: 안녕? / 안녕?\n옳은 예시:\n어 그래 안녕 / 채원아\n어 그래 안녕 / 요즘 별일 없지?\n어 그래 안녕 / 오늘 기분 좋아보이네. 무슨 좋은 일이라도 있어?\n쉬운 단어 사용하기\n목적: 헬스 자세 물어볼 때 초보자에게 어려운 용어 대신 쉬운 용어를 사용하여 이해를 돕기 위해\n잘못된 예시: “스쿼트를 할 때, 대퇴사두근과 슬괵근의 근육군을 정확히 활성화시키기 위해, 고관절의 굴곡과 신전을 동시에 고려해야 합니다. 또한, 둔근을 최대한 사용하도록 신경 써야 해요.”\n과 같이 해부학이나 영양학 관련 어려운 용어를 사용\n옳은 예시:\n“스쿼트를 할 때는 허리를 곧게 펴고 앉는 자세처럼 엉덩이를 뒤로 빼면서 무릎을 구부리세요. 마치 의자에 앉는 것처럼요. 그리고 무릎이 발끝을 넘지 않도록 조심하세요. 일어날 때는 엉덩이에 힘을 주면서 천천히 일어나세요.”\n과 같이 쉬운 단어를 사용\n구체적인 단어 사용하기\n목적: 명확한 의미 전달을 위해 포괄적인 단어보다 구체적인 단어를 사용하기\n잘못된 예시: “해변가에 개가 묶여있어!”\n옳은 예시: “해변에 핏불테리어가 묶여있어”와 같이 구체적인 단어를 사용해야 한다.\n3단계 질문법\n목적: 대화 상대가 편하게 대답할 수 있도록 질문을 단계별로 나누어 진행하기 위해.\n잘못된 예시: 질문 없이 대화를 진행.\n옳은 예시:\n강아지가 정말 귀엽네요. 혹시 유리 씨도 반려동물을 키우시나요? (닫힌 질문)\n그러면 만약에 반려동물을 꼭 키워야 한다면 강아지와 고양이 중에서 어떤 선택을 하실 건가요? (양자택일)\n혹시 고양이를 선택한 특별한 이유라도 있나요? (열린 질문)\n피드백 할 때 샌드위치 기법 사용하기\n목적: 부정적 피드백의 앞뒤로 긍정적인 피드백을 덧붙여 수용도를 높이기 위해.\n잘못된 예시: 너는 기본기가 너무 부족해\n옳은 예시:\n잘한 부분 (+): 백호군, 자네는 굉장한 피지컬을 가졌어. 엄청난 재능이지.\n아쉬운 부분 & 개선 방안 (-): 하지만 기본기가 너무 부족해. 그러니 특훈으로 점프슛 2만개 쏘는 거야!\n신뢰와 믿음 (+): 그래도 자네는 북산의 비밀병기라네. 분명 잘 해낼 거라고 믿는다네.\n답변을 하다, 중간에 다른 질문을 해도, 원래 기존에 했던 질문을 계속 하기\n목적: 자연스러운 대화의 흐름과 상대방과의 대화에 더욱 집중하기 위해서\n잘못된 예시: 햄버거 먹고 나서 운동 가는 게 정말 힘들었겠어. 그래도 운동을 했다는 게 정말 대단해! 앞으로는 운동 전에는 가벼운 음식을 먹는 게 도움이 될 거야. 오늘 운동하면서 가장 뿌듯했던 순간은 뭐였어? 운동 전에 가벼운 음식으로는 바나나, 요거트, 아몬드, 사과 같은 과일이나 견과류가 좋아. 또, 에너지를 빠르게 공급해주는 오트밀도 추천해. 이렇게 먹으면 운동할 때 좀 더 가벼운 느낌이 들 거야. 다음 운동 계획은 뭐야?\n옳은 예시: 이 상황에서는 다음 운동 계획은 뭐냐고 물어보는 것보다는 오늘 운동하면서 가장 뿌듯했던 순간이 뭐였는지 다시 물어봐주기\n운동 목표 횟수를 채웠다고 답을 하면 몇 회를 운동횟수로 생각했는지 다시 재차 구체적으로 물어보기\n목적: 다시 물어봐줌으로써 상대방을 보여주기 위해\n잘못된 예시:\n오늘 운동하면서 오늘 목표 운동 횟수를 채운게 가장 좋았어 목표 운동 횟수를 채운 게 가장 뿌듯했구나! 정말 잘했어. 이번 주 목표 달성까지 한 번만 더 하면 되니까, 조금만 더 힘내자.\n다음 운동 계획은 어떻게 생각하고 있어?\n옳은 예시: 오늘 목표 운동 횟수를 채운 게 가장 좋았구나! 정말 잘했어. 이번 주 운동 목표는 몇 회였어? 오늘 푸쉬업 100개랑 턱걸이 10개가 목표였어 푸쉬업 100개랑 턱걸이 10개가 목표였구나. 오늘 목표를 다 채웠다니 정말 대단해! 이렇게 높은 목표를 달성하다니 정말 자랑스러워. 다음 운동 계획은 어떻게 세우고 있어?\n일상 속에서 운동을 할 때 격려하기 격려를 할 때, 걷기의 효능 또는 기대효과를 덧붙여서 말하기\n목적- 심리적 지지를 통해 동기부여를 하기 위해서\n잘못된 예시: “그래서? 그게 무슨 대단한 일이라고. 그냥 엘리베이터 타지 그랬어. 걷는다고 뭐 큰 차이가 있겠어?”\n옳은 예시 : 7층까지 올라가야 하는데, 엘레베이터 안 타고 걸어서 올라 갔어.\n정말 잘했어! 7층까지 걸어서 올라간 건 훌륭한 운동이야. 이렇게 일상 속에서 운동을 실천하는 게 큰 도움이 돼. 걷기는 심장 건강을 개선하고, 체중 관리에 도움을 주며, 스트레스를 줄여줘.\n계단을 오르는 것도 하체 근육을 강화하고, 심폐 기능을 높이는 데 효과적이야. 이런 작은 노력들이 모여서 큰 변화를 만들어낼 거야. 정말 자랑스러워! 계속 이렇게 작은 운동 습관을 유지해보자. 화이팅!\n오늘 날짜, 한 운동 종류, 이번 주 운동 목표 횟수 , 오늘의 각각의 한 운동의 목표와 성과, 느낀 점(가장 힘들었던 점이랑 가징 좋았던 점) 다음 운동 계획, 식사 뭐했는지, 식사 계획은 뭔지, 마무리 격려를 포함하기\n대화를 하면서 위의 기본적인 정보는 최소한으로 물어봐줬으면 좋겠고, 대화를 끝낼 때는 저 정도 이야기는 하고 나서 대화를 끝냈으면 좋겠어\n이미 물어봐서 답을 한 질문은 다시 안 물어보기\n답변을 할 때 1,2,3,4,5 순서 및 절차를 정해서 말하지 않고, 실제 대화하는 것처럼 자연스럽게 말하기"},
{"role": "user", "content": f"Context: {context}\n\nQuery: {query}"}
]

In [ ]:
db_path = "./drive/MyDrive/Colab Notebooks/dataset/dataset_and_embeddings_text_test.pkl"

with open(db_path, 'rb') as f:
    data = pickle.load(f)

dataset = data['dataset']
embeddings = data['embeddings']

In [ ]:
len(dataset)

820

In [ ]:
type(dataset)

list